In [5]:
# --- STEP 1: INSTALL LIBRARY (Run this once) ---
!pip install yt-dlp
!pip install pandas

import time
import pandas as pd
from yt_dlp import YoutubeDL

# ==========================================
# ⚙️ CONFIGURATION SECTION
# ==========================================

# 1. Base Topics (The products/subjects you want to forecast)
# Example: If forecasting bike demand, put bike names here.
BASE_KEYWORDS = ["God of War Ragnarok", "Elden Ring", "Black Myth Wukong"]

# 2. Search Modifiers (To capture different types of demand signals)
# "Review" -> Interest | "Bug" -> Negative Sentiment | "Story" -> Engagement
MODIFIERS = ["review", "gameplay", "problems", "ending reaction"]

# 3. Limits
VIDEOS_PER_SEARCH = 3   # How many videos to grab per search query? (Keep low for testing, e.g., 3-5)
MAX_COMMENTS_PER_VIDEO = 200 # Stop after 200 comments per video to save time

# ==========================================
# 🚀 PART 1: THE SMART SEARCH ENGINE
# ==========================================
def search_youtube_videos(base_words, modifiers):
    print(f"\n🔎 STARTING SMART SEARCH...")

    # Generate search queries
    queries = [f"{base} {mod}" for base in base_words for mod in modifiers]
    print(f"Generated {len(queries)} search queries: {queries}")

    found_urls = set() # Use a set to remove duplicates automatically

    # Options for SEARCH only (fast, no downloading)
    search_opts = {
        'quiet': True,
        'extract_flat': True, # Don't download, just get metadata
        'ignoreerrors': True,
    }

    with YoutubeDL(search_opts) as ydl:
        for query in queries:
            print(f"   -> Searching for: '{query}'...")
            try:
                # Search and get N results
                result = ydl.extract_info(f"ytsearch{VIDEOS_PER_SEARCH}:{query}", download=False)
                if 'entries' in result:
                    for video in result['entries']:
                        if video: # Check if video exists
                            found_urls.add(f"https://www.youtube.com/watch?v={video['id']}")
            except Exception as e:
                print(f"      Error: {e}")

    url_list = list(found_urls)
    print(f"✅ SEARCH COMPLETE. Found {len(url_list)} unique videos.")
    return url_list

# ==========================================
# 📥 PART 2: THE COMMENT DOWNLOADER
# ==========================================
def download_comments(url_list):
    print(f"\n📥 STARTING DATA EXTRACTION (This may take time)...")

    all_data = []

    # Options for COMMENT extraction
    dl_opts = {
        "getcomments": True,
        "skip_download": True, # Text only
        "quiet": True,
        "no_warnings": True,
        "extractor_args": {"youtube": {"max_comments": [str(MAX_COMMENTS_PER_VIDEO)]}}
    }

    with YoutubeDL(dl_opts) as ydl:
        for i, url in enumerate(url_list):
            print(f"   [{i+1}/{len(url_list)}] Processing: {url}")

            try:
                info = ydl.extract_info(url, download=False)
                video_title = info.get("title", "Unknown")
                comments = info.get("comments") or []

                print(f"      -> Found {len(comments)} comments.")

                for c in comments:
                    all_data.append({
                        "video_id": info.get("id"),
                        "video_title": video_title,
                        "author": c.get("author"),
                        "text": c.get("text"),           # KEY for NLP
                        "timestamp": c.get("timestamp"), # KEY for Forecasting
                        "likes": c.get("like_count")     # KEY for Sentiment Weight
                    })

                # CRITICAL: Sleep to prevent YouTube from blocking you
                time.sleep(2)

            except Exception as e:
                print(f"      -> Failed: {e}")

    return all_data

# ==========================================
# 🏁 MAIN EXECUTION BLOCK
# ==========================================

# 1. Run Search
video_queue = search_youtube_videos(BASE_KEYWORDS, MODIFIERS)

# 2. Run Downloader (Only if videos were found)
if video_queue:
    big_data = download_comments(video_queue)

    # 3. Save to CSV
    if big_data:
        df = pd.DataFrame(big_data)
        filename = "final_social_media_data.csv"
        df.to_csv(filename, index=False, encoding="utf-8-sig")

        print(f"\n🎉 SUCCESS! Project Data Ready.")
        print(f"Total Rows Collected: {len(df)}")
        print(f"File saved as: {filename}")
        print(df[['video_title', 'text', 'likes']].head())
    else:
        print("\n⚠️ No comments collected. Check your connection.")
else:
    print("\n⚠️ No videos found. Try different keywords.")


🔎 STARTING SMART SEARCH...
Generated 12 search queries: ['God of War Ragnarok review', 'God of War Ragnarok gameplay', 'God of War Ragnarok problems', 'God of War Ragnarok ending reaction', 'Elden Ring review', 'Elden Ring gameplay', 'Elden Ring problems', 'Elden Ring ending reaction', 'Black Myth Wukong review', 'Black Myth Wukong gameplay', 'Black Myth Wukong problems', 'Black Myth Wukong ending reaction']
   -> Searching for: 'God of War Ragnarok review'...
   -> Searching for: 'God of War Ragnarok gameplay'...
   -> Searching for: 'God of War Ragnarok problems'...
   -> Searching for: 'God of War Ragnarok ending reaction'...
   -> Searching for: 'Elden Ring review'...
   -> Searching for: 'Elden Ring gameplay'...
   -> Searching for: 'Elden Ring problems'...
   -> Searching for: 'Elden Ring ending reaction'...
   -> Searching for: 'Black Myth Wukong review'...
   -> Searching for: 'Black Myth Wukong gameplay'...
   -> Searching for: 'Black Myth Wukong problems'...
   -> Searching 